# Setup

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [4]:
from src.vfm.connection import Connection
from src.vfm.preprocessor import Preprocessor
import pandas as pd
from src.vfm.model.reg.gradient_boost import GradientBoost
from src.vfm.model.nn.swt_tft import SWTTFTModel

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\pytorch_forecasting\models\base\_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
connection = Connection()
# Get client
client = connection.get_client()

In [98]:
well = 'W06'
df_raw = connection.get_data(client, well = well, start=pd.Timestamp('2019-01-01'), end=pd.Timestamp('2023-01-31'))
df_raw.shape

CogniteProjectAccessError: You don't have access to the requested CDF project='production-optimisation-rd', reason: Unauthorized. The token may have expired or it was invalid. | code: 401 | X-Request-ID: e4f1ba91-49b0-97e0-8021-77afa55c1630 | cluster: api

In [7]:
oldest_ts_per_column = df_raw.notna().apply(lambda col: col.index[col].min())
oldest_ts_per_column

whp            2019-01-30 18:22:00
wht            2019-01-30 18:22:00
dhp            2019-01-30 18:22:00
dht            2019-01-30 18:22:00
choke          2019-01-30 18:22:00
dcp            2019-01-30 18:22:00
qo_well_test   2019-01-30 20:39:00
qg_well_test   2019-01-30 20:39:00
qw_well_test   2019-01-30 20:39:00
qo_mpfm        2019-01-30 18:22:00
qg_mpfm        2019-01-30 18:22:00
wc_mpfm        2019-01-30 18:22:00
dtype: datetime64[ns]

In [8]:
latest_ts_per_column = df_raw.notna().iloc[::-1].idxmax()
latest_ts_per_column

whp            2023-01-17 11:19:00
wht            2023-01-17 11:19:00
dhp            2023-01-17 11:19:00
dht            2023-01-17 11:19:00
choke          2023-01-17 11:19:00
dcp            2023-01-17 11:19:00
qo_well_test   2023-01-17 20:24:00
qg_well_test   2023-01-17 20:24:00
qw_well_test   2023-01-17 20:24:00
qo_mpfm        2023-01-17 11:19:00
qg_mpfm        2023-01-17 11:19:00
wc_mpfm        2023-01-17 11:19:00
dtype: datetime64[ns]

In [ ]:
timestamps = df_raw.index[df_raw["qo_well_test"].notna()].to_list()
timestamps

[Timestamp('2019-01-30 20:39:00'),
 Timestamp('2019-01-31 04:57:00'),
 Timestamp('2019-01-31 13:30:00'),
 Timestamp('2019-03-02 15:21:00'),
 Timestamp('2019-03-04 08:21:00'),
 Timestamp('2019-03-10 17:42:00'),
 Timestamp('2019-04-14 16:54:00'),
 Timestamp('2019-04-15 16:55:00'),
 Timestamp('2019-04-15 23:44:00'),
 Timestamp('2019-04-16 07:08:00'),
 Timestamp('2019-05-22 17:07:00'),
 Timestamp('2019-05-24 05:19:00'),
 Timestamp('2019-05-24 07:50:00'),
 Timestamp('2019-05-24 10:48:00'),
 Timestamp('2019-05-24 17:02:00'),
 Timestamp('2019-05-25 20:16:00'),
 Timestamp('2019-05-26 05:12:00'),
 Timestamp('2019-05-26 06:59:00'),
 Timestamp('2019-05-26 10:20:00'),
 Timestamp('2019-05-26 18:57:00'),
 Timestamp('2019-05-27 01:57:00'),
 Timestamp('2019-05-27 08:01:00'),
 Timestamp('2019-07-05 09:49:00'),
 Timestamp('2019-07-05 22:05:00'),
 Timestamp('2019-08-27 21:52:00'),
 Timestamp('2019-08-28 01:12:00'),
 Timestamp('2019-08-28 06:02:00'),
 Timestamp('2019-08-28 08:06:00'),
 Timestamp('2019-09-

In [244]:
preprocessor = Preprocessor(df=df_raw)
df = preprocessor.preprocess_timeseries()
df.shape

(2085243, 11)

In [ ]:
df_raw.to_csv("output.csv", index=True)

In [183]:
independent_vars=["dhp", "dht", "whp", "wht", "choke", "dcp"]
summary = df[independent_vars].agg(['min', 'max', 'count'])
print(summary)

                dhp           dht           whp           wht         choke  \
min    0.000000e+00  0.000000e+00  2.557475e+06  5.416870e+01  2.550000e-01   
max    1.635270e+07  8.191725e+01  5.659456e+06  6.485973e+01  9.999885e-01   
count  2.085243e+06  2.085243e+06  2.085243e+06  2.085243e+06  2.085243e+06   

                dcp  
min    2.083724e+06  
max    2.287057e+06  
count  2.085243e+06  


In [223]:
# Count NaNs per column
nan_counts = df[df.columns.to_list()].isna().sum()

print(nan_counts)

dhp                   0
dht                   0
whp                   0
wht                   0
choke                 0
dcp                   0
qo_well_test    2085004
qg_well_test    2085003
qw_well_test    2085102
well_id               0
time_idx              0
dtype: int64


In [188]:
df.to_csv("output_filled.csv", index=True)

In [167]:
len(df_raw)

2085243

# Gradient Boost

In [238]:
dependent_vars=["qo_well_test", "qg_well_test", "qw_well_test"]
independent_vars=["dhp", "dht", "whp", "wht", "choke", "dcp"]
df_target = df.dropna(subset=["qo_well_test", "qg_well_test", "qw_well_test"])
df_target.shape

(141, 11)

In [245]:
gb_model = GradientBoost(dependent_vars=dependent_vars, independent_vars=independent_vars, df_target=df_target)

In [246]:
gb_model.train()

NameError: name 'XGBRegressor' is not defined

In [231]:
df_dense = gb_model.generate_dense_well_rates(df=df)

In [232]:
df_dense

,dhp,dht,whp,wht,choke,dcp,qo_well_test,qg_well_test,qw_well_test,well_id,time_idx,qo_well_test_imputed,qg_well_test_imputed,qw_well_test_imputed
2019-01-30 18:22:00,1.512291e+07,81.757781,4.546175e+06,60.786572,0.623996,2.234044e+06,154.637978,19442.161696,5.678685,W06,0,False,False,False
2019-01-30 18:23:00,1.512291e+07,81.757781,4.546175e+06,60.786572,0.623996,2.234044e+06,154.637978,19442.161696,5.678685,W06,1,False,False,False
2019-01-30 18:24:00,1.512291e+07,81.757781,4.546175e+06,60.786572,0.623996,2.234044e+06,154.637978,19442.161696,5.678685,W06,2,False,False,False
2019-01-30 18:25:00,1.512291e+07,81.757781,4.546175e+06,60.786572,0.623996,2.234044e+06,154.637978,19442.161696,5.678685,W06,3,False,False,False
2019-01-30 18:26:00,1.512291e+07,81.757781,4.546175e+06,60.786572,0.623996,2.234044e+06,154.637978,19442.161696,5.678685,W06,4,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-17 20:20:00,0.000000e+00,0.000000,2.613663e+06,59.534709,0.999416,2.146505e+06,101.353582,14908.457500,9.048504,W06,2085238,False,False,False
2023-01-17 20:21:00,0.000000e+00,0.000000,2.613663e+06,59.534709,0.999416,2.146505e+06,101.353582,14908.457500,9.048504,W06,2085239,False,False,False
2023-01-17 20:22:00,0.000000e+00,0.000000,2.613663e+06,59.534709,0.999416,2.146505e+06,101.353582,14908.457500,9.048504,W06,2085240,False,False,False
2023-01-17 20:23:00,0.000000e+00,0.000000,2.613663e+06,59.534709,0.999416,2.146505e+06,101.353582,14908.457500,9.048504,W06,2085241,False,False,False


# Sparse Well-Test Temporal Fusion Transformer Model - Well 06

In [ ]:
swt_tft_model = SWTTFTModel(dependent_variables=dependent_vars,
                            independent_variables=independent_vars,
                             df=df)

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_coun

AssertionError: filters should not remove entries all entries - check encoder/decoder lengths and lags

In [93]:
swt_tft_model.train()

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode  | FLOPs
--------------------------------------------------------------------------------------------------------
0  | loss                               | MultiLoss                       | 0      | train | 0    
1  | logging_metrics                    | ModuleList                      | 0      | train | 0    
2  | input_embeddings                   | MultiEmbedding                  | 1      | train | 0    
3  | prescalers                         | ModuleDict                      | 480    | train | 0    
4  | static_variable_selection          | VariableSelectionNetwork        | 14.0 K | train | 0    
5  | encoder_variable_selection         | VariableSel

<class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>


c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=21` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 496/521204 [01:14<21:49:57,  6.63it/s, v_num=2, train_loss_step=2124.5] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\perer\Documents\Code\pcperera\vfm\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import matplotlib.pyplot as plt

for well_idx in range(1):  # Plot all workload timeseries
    # for field_idx in range(len(target_fields)):
        fig, ax = plt.subplots(figsize=(10, 4))
        best_tft.plot_prediction(x=raw_predictions.x, out=raw_predictions.output, idx=well_idx, add_loss_to_title=False, ax=ax)
        current_title = ax.get_title()
        new_title = f' {well} qg_mpfm - known reals {time_varying_known_reals} \n' + current_title
        ax.set_title(new_title)